# DashStyle Fusion
#### Apply the style of one ArcGIS Dashboard to other ArcGIS Dashboards
created by Niklas Köhn, Esri Deutschland, 2024

In [27]:
from arcgis.gis import GIS
import getpass

# user input username + password
username = input("Username: ")
password = getpass.getpass(prompt='Password: ', stream=None) # hide password

# login to ArcGIS Online or ArcGIS Enterprise
gis = GIS("https://www.arcgis.com", username, password)
print("Successfully logged in as: " + gis.properties.user.username)

Username: esride-nik
Password: ········
Successfully logged in as: esride-nik


### Search for dashboards

In [ ]:
# user select to search org or own account
print("1. Search in the organization")
print("2. Search in my account")
choice = input("Enter your choice: ")

# search for dashboards in the org or own account
outside_org_choice = True
if choice == "1":
    outside_org_choice = False

search = input("Enter search term (empty string to search for all dashboards): ")
items = gis.content.search(query=search, item_type="Dashboard", max_items=100, outside_org=outside_org_choice)
print("Found " + str(len(items)) + " dashboards")

for item in items:
    print(item.title + " (" + item.id + ")")

1. Search in the organization
2. Search in my account


### Copy all dashboards to folder in own account

In [ ]:
import datetime

# create folder in own account named "dashboards_<date>_time"
folder_name = "dashboards_" + datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
folder = gis.content.create_folder(folder_name)

# copy all dashboards to the folder
copied_items = []
for item in items:
    # use arcgis python api to copy dashboard item. folder parameter does not work, everything ends up in root :(
    copied_item = item.copy_item(folder=folder_name)
    copied_items.append(copied_item)
    print ("Dashboard " + copied_item.title + " (" + copied_item.id + ")" + " copied.")

# workaround because folder parameter doesn't work: move items. if this fails, delete copies.
moved_items = []
deleted_items = []
for item in copied_items:
    objMoved = item.move(folder=folder_name)  #folder not found for given owner message if it doesnt move it.
    moved_items.append(item)
    if (objMoved is None) or \
            (objMoved is not None and 'success' in objMoved and not objMoved['success']):
        deleted_items.append(item)
        item.delete(force=False, dry_run=False)
        sys.exit(
            "ERROR: Unable to move the item with ItemID {0} to the folder, {1}. ".format(
                id, folder_name))
    
for item in moved_items:
    print ("Dashboard " + copied_item.title + " (" + copied_item.id + ")" + " moved to folder " + folder_name + ".")
print (str(len(moved_items)) + " dashboards copied in total.")

### Select template

In [ ]:
# user input dashboard id to use as template
template_id = input("Enter dashboard id to use as template: ")

# filter moved items to find template dashboard
template_item = None
for item in moved_items:
    if item.id == template_id:
        template_item = item
        break
print("template_item", template_item)

# get template dashboard JSON
template_json = template_item.get_data()

header = None
# copy header section from template_json
if template_json["version"] > 57:
    # TODO: check since when we have a "desktopView"
    header = template_json["desktopView"]["header"]
else:
    header = template_json["header"]

# copy theme property from template_json
theme = template_json["theme"]

# copy themeOverrides from template_json
themeOverrides = template_json["themeOverrides"]

print (header, "\n\n ... \n\n", theme, "\n\n ... \n\n", themeOverrides)

### Apply template header and theme

In [ ]:
for item in moved_items:
    # get template dashboard JSON
    edit_json = item.get_data()

    # copy header section from template_json
    if edit_json["version"] > 57:
        # TODO: check since when we have a "desktopView"
        edit_json["desktopView"]["header"] = header
    else:
        edit_json["header"] = header

    # copy theme property from template_json
    edit_json["theme"] = theme

    # copy themeOverrides from template_json
    edit_json["themeOverrides"] = themeOverrides
 
    # store item to ArcGIS Online or ArcGIS Enterprise
    update_success = item.update(data=edit_json)
    
    if update_success:
        print ("Item " + item.title + " (" + item.id + ") updated.")
    else:
        sys.exit("ERROR: Item {0} ({1}) could not be updated. ".format(item.title, item.id))